In [1]:
import numpy as np
import pandas as pd
from one.api import ONE
from psyfun import io
from psyfun.config import *
from psyfun import plots
import matplotlib as pl
from matplotlib import pyplot as plt

one = ONE()

# only run if first-time setup
# Fetching all session, insertion, and spike data
# DAVIDE: for some reason, fetching keeps getting interrupted by the absence of certain files/directories.
#%run -i fetchdata.py -a

# Load session and insertion metadata
df_sessions = pd.read_parquet(paths['sessions'])
df_insertions = pd.read_parquet(paths['insertions'])

# dropping na (PCA_DC)
df_sessions = df_sessions.query('n_tasks >= 2').dropna(subset=['task00_spontaneous_start', 'task01_spontaneous_start'])

# Create a DataFrame with eids (all sessions with a left vid), categorized by experimental group
eids_exp = df_sessions.loc[(df_sessions['raw_video_data/_iblrig_leftCamera.raw.mp4'] == True) & (df_sessions['control_recording'] == False), 'eid']
eids_cont = df_sessions.loc[(df_sessions['raw_video_data/_iblrig_leftCamera.raw.mp4'] == True) & (df_sessions['control_recording'] == True), 'eid']

In [2]:
import os
import numpy as np
import pandas as pd

def download_video(eid, camera = "left"):
    """
    Downloads raw video data from specific session. Camera parameter can be switched to "body" and "right".
    
    Parameters
    ----------
    eid (str): Experiment/session identifier

    camera (str): Camera orientation identifier (left by default). "body", "right" are other available options.

    Returns
    -------
    video_path: Local path for saved video
    """
    
    if camera == "left":
        video_path = one.load_dataset(eid, f'*leftCamera.raw*', collection='raw_video_data')
    elif camera == "right":
        video_path = one.load_dataset(eid, f'*rightCamera.raw*', collection='raw_video_data')
    elif camera == "body":
        video_path = one.load_dataset(eid, f'*bodyCamera.raw*', collection='raw_video_data')
    else:
         raise ValueError(f"Unknown camera parameter: {camera}")

    return video_path

def get_time_epochs(df_sessions, eid):

    video_data = df_sessions.loc[df_sessions['eid'] == eid]
    
    # epochs
    epoch_pre = [
        ("task00_spontaneous", video_data['task00_spontaneous_start'].values[0], video_data['task00_spontaneous_stop'].values[0]),
        ("task00_rfm", video_data['task00_rfm_start'].values[0], video_data['task00_rfm_stop'].values[0]),
        ("task00_replay", video_data['task00_replay_start'].values[0], video_data['task00_replay_stop'].values[0])
    ]

    epoch_post = [
        ("task01_spontaneous", video_data['task01_spontaneous_start'].values[0], video_data['task01_spontaneous_stop'].values[0]),
        ("task01_rfm", video_data['task01_rfm_start'].values[0], video_data['task01_rfm_stop'].values[0]),
        ("task01_replay", video_data['task01_replay_start'].values[0], video_data['task01_replay_stop'].values[0])
    ]

    return epoch_pre, epoch_post


# will be useful for multi-video analysis at the end
def load_pca_results(eid):
    """
    Imports FaceMap results on motion energy video analysis. Obtains path for .npy file based on eid, loads, and returns the data
    in a dictionary dataframe.

    Parameters
    ----------
        eid (str): Experiment/session identifier.

    Returns
    -------
        dictionary: Loaded PCA results.
        Keys() : (parameters )'filenames', 'save_path', 'Ly', 'Lx', 'sbin', 'fullSVD', 'save_mat', 'Lybin', 'Lxbin', 'sybin', 'sxbin', 'LYbin', 'LXbin',
                 (outputs) 'avgframe', 'avgmotion', 'avgframe_reshape', 'avgmotion_reshape', 'motion', 'motSv', 'movSv', 'motMask', 'movMask', 'motMask_reshape', 'movMask_reshape', 'motSVD', 'movSVD', 
                (unused tracking) 'pupil', 'running', 'blink', 'rois', 'sy', 'sx']
    """
    # Assuming consistent naming
    file_path = os.path.join(one.eid2path(eid), "raw_video_data", "_iblrig_leftCamera.raw_proc.npy")
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"PCA results file not found: {file_path}")
    

    data = np.load(file_path, allow_pickle=True).item()
    return data


In [3]:
#for eid in eids_cont:
 #   video_path = download_video(eid)

(S3) C:\Users\marti\Downloads\ONE\alyx.internationalbrainlab.org\mainenlab\Subjects\ZFM-08457\2025-03-11\001\raw_video_data\_iblrig_leftCamera.raw.mp4: 100%|██████████| 4.80G/4.80G [07:47<00:00, 10.3MB/s]  
(S3) C:\Users\marti\Downloads\ONE\alyx.internationalbrainlab.org\mainenlab\Subjects\ZFM-08457\2025-03-18\001\raw_video_data\_iblrig_leftCamera.raw.mp4: 100%|██████████| 4.14G/4.14G [06:39<00:00, 10.4MB/s]
(S3) C:\Users\marti\Downloads\ONE\alyx.internationalbrainlab.org\mainenlab\Subjects\ZFM-08458\2025-03-11\001\raw_video_data\_iblrig_leftCamera.raw.mp4: 100%|██████████| 4.39G/4.39G [06:49<00:00, 10.7MB/s]
(S3) C:\Users\marti\Downloads\ONE\alyx.internationalbrainlab.org\mainenlab\Subjects\ZFM-08458\2025-03-14\001\raw_video_data\_iblrig_leftCamera.raw.mp4: 100%|██████████| 4.64G/4.64G [08:20<00:00, 9.26MB/s]
(S3) C:\Users\marti\Downloads\ONE\alyx.internationalbrainlab.org\mainenlab\Subjects\ZFM-08584\2025-03-12\001\raw_video_data\_iblrig_leftCamera.raw.mp4: 100%|██████████| 4.67G/4.6

In [3]:
def expected_path(video_path, start_sec, end_sec):
    """
    Constructs expected path to check if epoch was already processed.
    """
    basename, filename = os.path.split(video_path)
    filename, _ = os.path.splitext(filename)
    epoch_str = f"_start{int(start_sec)}_end{int(end_sec)}"
    expected_save = os.path.join(basename, f"{filename}{epoch_str}_proc.npy")

    return expected_save


In [4]:
import motionmap
import ibllib.io.video as vidio

results = []  # List to store (eid, epoch, savename)

for eid in eids_cont:
    # epoch timing 
    epoch_pre, epoch_post = get_time_epochs(df_sessions, eid)

    # Get video path
    url = one.eid2path(eid)
    if not os.path.exists(url):
        video_path = download_video(eid)
    
    video_path = os.path.join(url, "raw_video_data", "_iblrig_leftCamera.raw.mp4")
    path = [[video_path]]
    
    meta = vidio.get_video_meta(video_path)
    fps = meta['fps']

    # Run for each pre-epoch
    for epoch_name, start_sec, end_sec in epoch_pre:
        # expected save path
        expected_save = expected_path(video_path, start_sec, end_sec) 
        if os.path.exists(expected_save):
            print(f"Skipping {eid} pre_{epoch_name}: already processed at {expected_save}")
            results.append((eid, f'pre_{epoch_name}', expected_save))
            continue

        print("Running session: ", eid, " - epoch: ", epoch_name)
        savename = motionmap.run(path, sbin=4, start_sec=start_sec, end_sec=end_sec)
        results.append((eid, f'pre_{epoch_name}', savename))

    # Run for each post-epoch
    for epoch_name, start_sec, end_sec in epoch_post:
        expected_save = expected_path(video_path, start_sec, end_sec) 
        if os.path.exists(expected_save):
            print(f"Skipping {eid} post_{epoch_name}: already processed at {expected_save}")
            results.append((eid, f'post_{epoch_name}', expected_save))
            continue

        print("Running session: ", eid, " - epoch: ", epoch_name)
        savename = motionmap.run(path, sbin=4, start_sec=start_sec, end_sec=end_sec)
        results.append((eid, f'post_{epoch_name}', savename))

Skipping c7cf8e25-1e2c-4b03-a5f5-5a049f1cd228 pre_task00_spontaneous: already processed at C:\Users\marti\Downloads\ONE\alyx.internationalbrainlab.org\mainenlab\Subjects\ZFM-08457\2025-03-11\001\raw_video_data\_iblrig_leftCamera.raw_start0_end300_proc.npy
Skipping c7cf8e25-1e2c-4b03-a5f5-5a049f1cd228 pre_task00_rfm: already processed at C:\Users\marti\Downloads\ONE\alyx.internationalbrainlab.org\mainenlab\Subjects\ZFM-08457\2025-03-11\001\raw_video_data\_iblrig_leftCamera.raw_start300_end956_proc.npy
Skipping c7cf8e25-1e2c-4b03-a5f5-5a049f1cd228 pre_task00_replay: already processed at C:\Users\marti\Downloads\ONE\alyx.internationalbrainlab.org\mainenlab\Subjects\ZFM-08457\2025-03-11\001\raw_video_data\_iblrig_leftCamera.raw_start956_end1269_proc.npy
Skipping c7cf8e25-1e2c-4b03-a5f5-5a049f1cd228 post_task01_spontaneous: already processed at C:\Users\marti\Downloads\ONE\alyx.internationalbrainlab.org\mainenlab\Subjects\ZFM-08457\2025-03-11\001\raw_video_data\_iblrig_leftCamera.raw_start3

In [5]:
results_df = pd.DataFrame(results)
results_df.columns = ["eid", "epoch", "savename"]

results_df.to_csv('cont_results', index=False)